In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
n=10   # number of inputs

beta=np.zeros(n) ## first and last parameters are 1, others are zeros
beta[1]=1   
beta[-1]=1
print(beta)
beta.shape=(n,1)
print(beta)
print("-----")
x=np.random.rand(1,n)[0]
print("x: ", x)
x_T_beta = x.dot(beta)
w = 0.1*np.random.rand(1)[0]
print("x_t_beta : ", x_T_beta)
print("w : ", w)
#y=float(x.dot(beta))+0.1*np.random.rand(1)[0] ## y =x^T beta +w
y=float(x_T_beta)+w ## y =x^T beta +w
print("y : ", y)
message=np.array2string(np.append([0,y],x),separator=",") 
print(message)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]
-----
x:  [0.49371378 0.63800579 0.77470965 0.64438818 0.19670207 0.07308795
 0.06979703 0.30618327 0.66376063 0.66920123]
x_t_beta :  [1.30720702]
w :  0.07222462960787528
y :  1.3794316541583609
[0.        ,1.37943165,0.49371378,0.63800579,0.77470965,0.64438818,
 0.19670207,0.07308795,0.06979703,0.30618327,0.66376063,0.66920123]


In [3]:
def prepareDataFrame(data_file):
    #Takes about one minute to load
    data=pd.read_csv(data_conv,header=None,sep=" ")
    data.columns=["Date","Hour","Sensor","Value","Voltage"]
    data=data.sort_values(['Date','Hour']).reset_index(drop=True)
    
    data['datetime']=pd.to_datetime(data.Date+' '+data.Hour)
    data['relative_datetime']=data['datetime']-data['datetime'][0]
    data['seconds']=data['relative_datetime'].dt.total_seconds()
    
    sensorId_type=data.Sensor.str.split("-",expand=True)
    sensorId_type.columns=['SensorId','Type']
    data['SensorId']=sensorId_type['SensorId'].astype(int)
    data['Type']=sensorId_type['Type'].astype(int)
    
    #Drop features not needed for the simulation
    data=data.drop(['datetime','relative_datetime','Sensor','Date','Hour','Voltage'],axis=1)
    return data

In [4]:
data_conv = "../data/data.conv.txt"
data = prepareDataFrame(data_conv)

In [71]:
temp_1=data[(data.SensorId==1) & (data.Type==0) & (data.seconds<=8*86400)]
temp_1=temp_1.reset_index(drop=True)

In [72]:
temp_24=data[(data.SensorId==24) & (data.Type==0) & (data.seconds<=8*86400)]
temp_24=temp_24.reset_index(drop=True)

In [73]:
# 5 closest neighbors of sensor 1 are sensors 2, 3, 33, 34, 35
neighbors_1 = [data[(data.SensorId==2) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==3) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==33) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==34) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==35) & (data.Type==0) & (data.seconds<=8*86400)]]

In [74]:
# 5 closest neighbors of sensor 24 are sensors 22, 23, 25, 26, 27
neighbors_24 = [data[(data.SensorId==22) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==23) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==25) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==26) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==27) & (data.Type==0) & (data.seconds<=8*86400)]]

In [111]:
def convertTimeToSlots(dataframe):
    interval_slot = 30
    #divide data in slots of 30sec, add each slot value to each entry
    dataframe["slot"] = (dataframe["seconds"]//interval_slot).astype(int)
    #transform the seconds so that for each slot, its corresponding 'seconds' value is at the center of this slot (usefull for plots)
    dataframe["seconds"] = interval_slot*(dataframe["slot"] + dataframe["slot"]+1) / 2
    
    #Take care of the potential multiple value appearing within the same slot -> average them
    dataframe = dataframe.groupby(["slot"]).agg("mean")
    dataframe.reset_index(level=0, inplace=True)
    return dataframe

def fillMissingRows(dataframe):
    interval_slot = 30
    missing_values = {"slot": [], "seconds": [], "SensorId": [], "Type": []}
    for i in range (23040): #total nb of slots for 8 days
        if i not in dataframe["slot"].values:
            seconds = interval_slot*(i + i+1) / 2
            missing_values["slot"].append(i)
            missing_values["seconds"].append(seconds)
            missing_values["SensorId"].append(1)
            missing_values["Type"].append(0)
    #Build DataFrame with missing values
    temp_missing = pd.DataFrame(missing_values)
    #Merge the two Dataframe and sort them by values of the 'slot' column
    #At this point, the temperature values are still missing -> NaN
    complete_temp = dataframe.append(temp_missing).sort_values('slot')
    #Replace NaN by values extracted from a linear method based on the neighbors
    complete_temp["Value"] = complete_temp["Value"].interpolate(limit_direction="both")    
    return complete_temp

def preprocessDataFrames(output_df, input_dfs):
    output_df = fillMissingRows(convertTimeToSlots(output_df))
    for i in range (len(input_dfs)):
        input_dfs[i] = fillMissingRows(convertTimeToSlots(input_dfs[i]))
    return output_df, input_dfs


In [112]:
temp_1, neighbors_1 = preprocessDataFrames(temp_1, neighbors_1)    

print(temp_1)


/home/kais/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


       SensorId  Type      Value   seconds   slot
0             1     0  19.243600      15.0      0
1             1     0  19.243600      45.0      1
0             1     0  19.243600      75.0      2
2             1     0  19.237067     105.0      3
3             1     0  19.230533     135.0      4
1             1     0  19.224000     165.0      5
2             1     0  19.214200     195.0      6
4             1     0  19.199500     225.0      7
3             1     0  19.184800     255.0      8
5             1     0  19.189700     285.0      9
4             1     0  19.194600     315.0     10
6             1     0  19.184800     345.0     11
5             1     0  19.175000     375.0     12
6             1     0  19.184800     405.0     13
7             1     0  19.184800     435.0     14
7             1     0  19.184800     465.0     15
8             1     0  19.175000     495.0     16
8             1     0  19.168467     525.0     17
9             1     0  19.161933     555.0     18


In [16]:
def applyRLS(output_df, inputs_df):
    nb_features = len(inputs_df)
    betas = np.zeros(nb_features)
    V = np.diag(np.zeros(n)+10)
    forgetting_factor = 1.0
    

            beta=state[1]
        beta.shape=(n,1)
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]   ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T))))
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        
        return (key,beta,V,mu,sse/(N+1.0),N+1)  ## update formula mod1
    
#def mergeByslots(temp_data):
#def mapSecondsToSlots(temperature_data, neighbors_data):
#    interval_slot = 30  #30s slots
#    nb_slots = int(86400/interval_slot)
#    print(nb_slots)
#    for day in range (0,8):
        

#def predictSlots(temperature_data, neighbors_data):
#    interval_slot = 30  #30s slots
#    nb_slots = int(86400/interval_slot)
#    print(nb_slots)
    
    


In [86]:
# Creating the first Dataframe using dictionary 
df1 = pd.DataFrame({"a":[1, 2, 3, 4], 
                    "b":[5, 6, 7, 8]}) 
print(df1)
# Creating the Second Dataframe using dictionary 
df2 = pd.DataFrame({"a":[1, 2, 3], 
                    "b":[5, 6, 7],  
                    "c":[1, 5, 4]}) 
print(df2)
  
# for appending df2 at the end of df1 
df = df1.append(df2, ignore_index = True) 
print(df)


   a  b
0  1  5
1  2  6
2  3  7
3  4  8
   a  b  c
0  1  5  1
1  2  6  5
2  3  7  4
   a  b    c
0  1  5  NaN
1  2  6  NaN
2  3  7  NaN
3  4  8  NaN
4  1  5  1.0
5  2  6  5.0
6  3  7  4.0
